<a href="https://colab.research.google.com/github/dareoyeleke/ACA_TRENDS/blob/main/aca_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
year22_plan = pd.read_csv('/content/plan_attributes_22.csv')
year23_plan = pd.read_csv('/content/plan_attributes_23.csv')
year24_plan = pd.read_csv('/content/plan_attributes_24.csv')
year25_plan = pd.read_csv('/content/plan_attributes_25.csv')
year26_plan = pd.read_csv('/content/plan_attributes_26.csv')


In [ ]:
cols_22 = set(year22_plan.columns)
cols_23 = set(year23_plan.columns)
cols_24 = set(year24_plan.columns)
cols_25 = set(year25_plan.columns)
cols_26 = set(year26_plan.columns)

In [ ]:
(cols_22 ^ cols_23) | (cols_23 ^ cols_24) | (cols_24 ^ cols_25) | (cols_25 ^ cols_26)


In [ ]:
for df in [year22_plan, year23_plan, year24_plan, year25_plan, year26_plan]:
    df.drop(columns=["DesignType"], errors="ignore", inplace=True)
year22_plan
common_cols = cols_22 & cols_23 & cols_24 & cols_25 & cols_26
len(common_cols)

In [ ]:
plans_all = pd.concat([year22_plan, year23_plan, year24_plan, year25_plan, year26_plan], ignore_index=True)

In [ ]:
plans_all.isna().sum().sort_values(ascending=False).head()

In [ ]:
plans_all = plans_all.rename(columns={
    "PlanId": "plan_id",
    "BusinessYear": "business_year",
    "IssuerMarketPlaceMarketingName": "issuer_name",
    "MetalLevel": "metal_level",
    "PlanType": "plan_type",
    "MarketCoverage": "market_type",
    "StateCode": "state_code"
})


In [ ]:
core_cols = [
    "plan_id",
    "business_year",
    "issuer_name",
    "metal_level",
    "plan_type",
    "market_type",
    "state_code"
]

plans_all = plans_all[core_cols]


In [ ]:
refined_plans_all = plans_all.copy()
refined_plans_all.to_csv('refined_plans_all.csv')

In [ ]:

rate_22 = pd.read_csv('/content/rate_22.csv')
rate_23 = pd.read_csv('/content/rate_23.csv')
rate_24 = pd.read_csv('/content/Rate_24.csv')
rate_25 = pd.read_csv('/content/Rate_25.csv')
rate_26 = pd.read_csv('/content/Rate_26.csv')

In [ ]:

rate_cols_22 = set(rate_22.columns)
rate_cols_23 = set(rate_23.columns)
rate_cols_24 = set(rate_24.columns)
rate_cols_25 = set(rate_25.columns)
rate_cols_26 = set(rate_26.columns)

len(rate_cols_26)

In [ ]:
# All datasets have the similar columns and therefore return an empty set
(rate_cols_22 ^ rate_cols_23) | (rate_cols_23 ^ rate_cols_24) |(rate_cols_24 ^ rate_cols_25) |(rate_cols_25 ^ rate_cols_26)

In [ ]:
all_rates = pd.concat([rate_22, rate_23, rate_24, rate_25, rate_26])

In [ ]:
all_rates.isna().sum().sort_values(ascending=False).head()

In [ ]:
all_rates['BusinessYear'].unique()

In [ ]:
core_rate_cols = [
    "PlanId",
    "BusinessYear",
    "StateCode",
    "RatingAreaId",
    "Age",
    "IndividualRate"
]

core_all_rates_df = all_rates[core_rate_cols].rename(columns={
    "PlanId": "plan_id",
    "BusinessYear": "plan_year",
    "StateCode": "state_code",
    "RatingAreaId": "rating_area_id",
    "Age": "age",
    "IndividualRate": "individual_rate"
})


In [ ]:
core_all_rates_df
core_all_rates_df['plan_year'].unique()

In [ ]:
core_all_rates_df.to_csv('core_all_rates_df.csv', index=False)

In [ ]:
core_all_rates_df

In [ ]:
rates_for_30 = core_all_rates_df[core_all_rates_df['age'] == '30']
rates_for_30.to_csv('rates_for_30.csv')
rates_for_30

In [ ]:
refined_plans_all = pd.read_csv('/content/refined_plans_all.csv')

In [ ]:
refined_plans_all = refined_plans_all.rename(columns={'business_year' : 'plan_year'})
refined_plans_all # has a high amount of duplicates I need to drop
all_plans_nodups = refined_plans_all.copy() # created a copy to keep the original intact
all_plans_nodups = all_plans_nodups.drop_duplicates(subset=['plan_id', 'plan_year', 'state_code']) # dropped duplicates from the main columns
all_plans_nodups.duplicated(['plan_id', 'plan_year', 'state_code']).sum() # checked for duplicates in main columns. 0 dups
all_plans_nodups.duplicated().sum() # checked for duplicates in whole table. 0 duplicates

In [ ]:
all_plans_nodups.to_csv('all_plans_no_dups.csv')

In [ ]:
rates_for_30['plan_year'].unique()
rates_for_30

In [ ]:
rates_agg.duplicated(['plan_id', 'plan_year', 'state_code']).sum() # zero duplicates
rates_agg.shape

In [ ]:
rates_agg = (
    rates_for_30
    .groupby(["plan_id", "plan_year", "state_code"], as_index=False)
    .agg(avg_premium=("individual_rate", "mean"))
)
rates_agg['plan_year'].unique()
# rates_agg.to_csv('rates_agg.csv')
rates_agg # helps to make unique rows per plan, year and state and give avg premium for each plan according to the individual rate associated with it

In [ ]:
plan_ids = set(all_plans_nodups['plan_id'].unique())
rates_ids = set(rates_agg['plan_id'].unique())
len(plan_ids & rates_ids), len(plan_ids), len(rates_ids)

In [ ]:
all_plans_nodups['plan_id'].head(), rates_agg['plan_id'].head()

In [ ]:
all_plans_nodups["plan_id"] = (
    refined_plans_all["plan_id"]
    .astype(str)
    .str.split("-", n=1)
    .str[0]
)
refined_plans_all

In [ ]:
plan_ids = set(all_plans_nodups['plan_id'].unique())
rates_ids = set(rates_agg['plan_id'].unique())
len(plan_ids & rates_ids), len(plan_ids), len(rates_ids)

In [ ]:
final_plans_rates = all_plans_nodups.merge(rates_agg, on=['plan_id', 'plan_year', 'state_code'], how='inner')
final_plans_rates

In [ ]:
final_plans_rates.duplicated(['plan_year', 'plan_id', 'state_code']).sum()
final_plans_rates.shape
final_plans_rates.to_csv('final_plans_rates.csv')
final_plans_rates['plan_year'].value_counts().sort_index()
final_plans_rates['metal_level'].value_counts()

In [ ]:
# average premium by year

yearly_avg_prem = final_plans_rates.groupby('plan_year', as_index=False).agg(avg_premium=('avg_premium', 'mean'))
yearly_avg_prem.plot(kind='line', x='plan_year', y= 'avg_premium')
plt.title("Average Premium By Year")
plt.xlabel('Premium Year')
plt.ylabel('Premium in USD ($)')
plt.xticks(rotation= 45)
plt.show()

In [ ]:
# avergae YoY change
yearly_avg_prem['YoY_pct_change'] = yearly_avg_prem['avg_premium'].pct_change() * 100
yearly_avg_prem

In [ ]:
avg_by_year_metal = final_plans_rates.groupby(['plan_year', 'metal_level'], as_index=False).agg(avg_premium=('avg_premium', 'mean'))

In [ ]:
avg_by_year_metal['YoY_percent_change'] = avg_by_year_metal.groupby("metal_level")["avg_premium"].pct_change() * 100
avg_by_year_metal

In [ ]:
latest_year = final_plans_rates['plan_year'].max()

avg_by_state_yrlatest = (final_plans_rates[final_plans_rates['plan_year'] == latest_year]
                         .groupby('state_code', as_index=False)
                         .agg(avg_premium =('avg_premium', 'mean'))
                         .sort_values('avg_premium', ascending=False))
avg_by_state_yrlatest # premiuims for 2026 by state starting with highest

In [ ]:
# state dispersion stats

state_dispersion = (final_plans_rates.groupby(['plan_year', 'state_code'])
                                              .agg(
                                                  min_premium = ('avg_premium', 'min'),
                                                  median_premium = ('avg_premium', 'median'),
                                                  max_premium = ('avg_premium', 'max')
                                              ))

state_dispersion['spread'] = (state_dispersion['max_premium'] - state_dispersion['min_premium'])

state_dispersion

In [ ]:
issuer_avg = (
    final_plans_rates
    .groupby(["plan_year", "issuer_name"], as_index=False)
    .agg(avg_premium=("avg_premium", "mean"))
)

top_issuers = (
    final_plans_rates["issuer_name"]
    .value_counts()
    .head(10)
    .index
)

issuer_avg_top = issuer_avg[
    issuer_avg["issuer_name"].isin(top_issuers)
]



In [ ]:
plans_by_year = (
    final_plans_rates
    .groupby("plan_year", as_index=False)
    .agg(plan_count=("plan_id", "nunique"))
)


In [ ]:
premium_dist = (
    final_plans_rates
    .groupby("plan_year")["avg_premium"]
    .describe()
)


In [ ]:
yearly_avg_prem.to_csv("metrics_avg_by_year.csv", index=False)
avg_by_year_metal.to_csv("metrics_avg_by_year_metal.csv", index=False)
avg_by_state_yrlatest.to_csv("metrics_avg_by_state_latest.csv", index=False)
issuer_avg_top.to_csv("metrics_issuer_avg_top.csv", index=False)
